In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader,Dataset,TensorDataset
import numpy as np

In [19]:
## Hyperparameters
device = torch.device('mps')
batch_size = 1024
epochs = 5
in_channel = 3
lr = 1e-3
num_classes = 10

In [27]:
## Load the Data


class DebugDataset(CIFAR10):
    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        if not isinstance(img, np.ndarray):
            img = np.array(img)
        return Image.fromarray(img), target

train_dataset = DebugDataset(
    root='/Users/abhishekwaghchaure/Desktop/Datasets',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)

NameError: name 'CIFAR10' is not defined

In [21]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /Users/abhishekwaghchaure/Desktop/Datasets
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [28]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity,self).__init__()

    def forward(self,x):
        return x

model = torchvision.models.vgg16(pretrained = True)
for param in model.parameters():
    param.requires_grad = False
print(model)
model.avgpool = Identity()
model.classifier = nn.Sequential(
    nn.Linear(512,100),
    nn.ReLU(),
    nn.Linear(100, num_classes)
)
print(model)
model.to(device = device)

/opt/anaconda3/envs/stable_env_nlp/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/stable_env_nlp/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [23]:
## Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optim = optim.Adam(model.parameters(), lr = lr)

In [26]:
model.train()
for epoch in range(epochs):
    loop = tqdm(enumerate(train_loader), total=len(train_loader))
    running_loss = 0.0  # Reset running loss at the start of each epoch
    for idx, (data, targets) in loop:
        # Move data and targets to the device
        data = data.to(device)
        targets = targets.to(device)
        
        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)
        running_loss += loss.item()  # Accumulate batch loss
        
        # Backward pass
        optim.zero_grad()  # Zero out the gradients
        loss.backward()  # Compute gradients
        
        # Optimization step
        optim.step()  # Update model parameters
        
        # Update tqdm loop display
        loop.set_description(f"Epoch = {epoch + 1}/{epochs}")
        loop.set_postfix(loss=loss.item())
    
    # Print epoch-level running loss
    print(f"Running Loss for the whole epoch: {running_loss / len(train_loader):.4f}")

  0%|          | 0/49 [00:00<?, ?it/s]


TypeError: expected np.ndarray (got numpy.ndarray)